In [1]:
from rapidfuzz import process, fuzz
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# Load your DataFrames
df_fnb = pd.read_csv('csv_data\fnb_list_part_2_cleaned.csv', encoding='utf-8')
df_halal = pd.read_csv('csv_data\premis_makanan_list_part_4_cleaned.csv', encoding='utf-8')

# Ensure data is in uppercase for consistent matching
df_fnb['Name'] = df_fnb['Name'].str.upper()
df_fnb['Address'] = df_fnb['Address'].str.upper()
df_halal['Premise Name'] = df_halal['Premise Name'].str.upper()
df_halal['Address'] = df_halal['Address'].str.upper()

THRESHOLD = 90
MAX_MATCHES = 3
matches = []

def match_row(fnb_row):
    fnb_name = fnb_row['Name']
    fnb_address = fnb_row['Address']
    match_results = []

    for _, halal_row in df_halal.iterrows():
        halal_name = halal_row['Premise Name']
        halal_address = halal_row['Address']
        name_score = fuzz.token_set_ratio(fnb_name, halal_name)
        address_score = fuzz.token_set_ratio(fnb_address, halal_address)

        if name_score >= THRESHOLD and address_score >= THRESHOLD:
            match_results.append((fnb_name, fnb_address, halal_name, name_score, halal_address, address_score))
            break  # Assuming you only want the first match above threshold

    return match_results

# Use ThreadPoolExecutor to parallelize the matching process
with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(match_row, fnb_row) for _, fnb_row in df_fnb.iterrows()]
    
    # Progress bar for futures
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Matching"):
        matches.extend(future.result())
        if len(matches) >= MAX_MATCHES:
            break

# Print the matches
for match in matches[:MAX_MATCHES]:
    print(f"FNB Name: {match[0]}, FNB Address: {match[1]}")
    print(f"Matched Halal Name: {match[2]} with score {match[3]}")
    print(f"Matched Halal Address: {match[4]} with score {match[5]}\n")


In [4]:
import pandas as pd
df_fnb = pd.read_csv('csv_data\fnb_list_part_2_cleaned.csv', encoding='utf-8')
df_fnb

,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url
0,#leleh,NaN,['Cafe'],Cheras,MyTOWN Shopping Centre,"Seksyen, 90, Jalan Cochrane, Cheras, 55100 Kua...",03-2720 1772,https://www.google.com/maps/place/%23leleh/dat...,NaN,NaN,NaN
1,1 Plus 1 Coffee • Fruit ( Café ),1 Plus 1 Coffee,['Cafe'],Puchong,NaN,"1, Jalan Puteri 7/11, Bandar Puteri, 47100 Puc...",016-934 7388,https://www.google.com/maps/place/1+Plus+1+Cof...,NaN,http://web.facebook.com/profile.pho?id=1000844...,https://grab.onelink.me/2695613898?pid=inappsh...
2,10 Pasento Puchong,10 Pasento Puchong,['Cafe'],Puchong,Floor 1 · Hotel Sri Sutra,"1st Floor, 10, Jalan Kenari 6, Bandar Puchong ...",NaN,https://www.google.com/maps/place/10+Pasento+P...,NaN,https://www.instagram.com/10_pasento/,NaN
3,103 Coffee,103 Coffee,['Cafe'],Seri Kembangan,NaN,"103-G, Jalan Radin Bagus, Bandar Baru Sri Peta...",012-519 7103,https://www.google.com/maps/place/103+Coffee/d...,http://103coffeeworkshop.beepit.com/,https://103coffee.com/,NaN
4,128 Thai,128 Thai,['Western Food'],Subang Jaya,NaN,"42, Jalan SS 15/4c, Ss 15, 47500 Subang Jaya, ...",016-233 0609,https://www.google.com/maps/place/128+Thai/dat...,NaN,https://128thai.business.site/?m=true,https://wa.me/message/EY3JJ7HNEGQ4G1
...,...,...,...,...,...,...,...,...,...,...,...
4065,麻婆婆麻辣香锅·冒菜（SS15店）,麻婆婆麻辣香锅,['Chinese Food'],Subang Jaya,NaN,"Selangor, Subang Jaya, 1nd Floor 70, Jalan SS ...",NaN,https://www.google.com/maps/place/%E9%BA%BB%E5...,NaN,NaN,https://www.foodpanda.my/restaurant/shtw/mapop...
4066,黄凳仔,黄凳仔,['Chinese Food'],Seri Kembangan,NaN,"13-25, Jalan BS 3/6, Taman Bukit Serdang, 4330...",NaN,https://www.google.com/maps/place/%E9%BB%84%E5...,NaN,NaN,NaN
4067,龍焱麻辣香锅 Loong Yan Mala Hotpot,龍焱麻辣香锅 Loong Yan Mala Hotpot,['Chinese Food'],Seri Kembangan,NaN,"8, Jalan PSK 4, Pusat Perdagangan Seri Kembang...",03-8958 5880,https://www.google.com/maps/place/%E9%BE%8D%E7...,https://wa.me//60389585880,NaN,https://www.foodpanda.my/restaurant/l4d2/loong...
4068,𝗖𝗵𝗮𝗺𝗽𝗶𝗴𝗻𝗼𝗻𝘀 𝗣𝗮𝘁𝗶𝘀𝘀𝗲𝗿𝗶𝗲,𝗖𝗵𝗮𝗺𝗽𝗶𝗴𝗻𝗼𝗻𝘀 𝗣𝗮𝘁𝗶𝘀𝘀𝗲𝗿𝗶𝗲,['Dessert'],Petaling Jaya,NaN,"27-1, Jalan PJU 5/11, Dataran Sunway, Kota Dam...",017-600 6710,https://www.google.com/maps/place/%F0%9D%97%96...,https://www.champignonspatisserie.com/en_MY/,https://www.champignonspatisserie.com/en_MY/,NaN
